In [ ]:
#from google.colab import drive
#drive.mount('/gdrive')

In [ ]:
#%cd ../gdrive/MyDrive/Kaggle/cassavaleaf
#!ls

In [ ]:
#! pip install -q kaggle
#! mkdir ~/.kaggle
#### ES necesario que tengas el token de kaggle on your gdrive directory
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json
#!kaggle datasets download -d tahsin/cassava-leaf-disease-merged

In [ ]:
#!mkdir kaggle_data
#!unzip "cassava-leaf-disease-merged.zip" -d kaggle_data

<center>
    <img src= "https://i0.wp.com/chichisgarden.com/wp-content/uploads/2020/10/Cassava-Plant.jpg?resize=650%2C325&ssl=1" width="800">
</center>


<center>
    <img src= "https://www.gstatic.com/devrel-devsite/prod/vc5f5097f7e98f45082257ed44f785e23f8176f944afb30dfad7aee218957f132/tensorflow/images/lockup.svg" width="200">
</center>
<h1 style="color:blue; font-family: 'Roboto', sans-serif; text-align:center; font-size:30px">Geting Started | Transfer Learning and Data Aumentatin 📚💬</h1>
<hr>

<h1 style="text-align:left; font-family: 'Roboto', sans-serif; "> ABOUT: </h1>

<h3 style="text-align:center">Hi everyone, the aims of this notebooks is show you how to achive a classication problem going trowgh data augmentation and pretrained models for transfer learning.</h3>

<h3 style="text-align:center">I'm going to start exploring the data and the type of data augmentation techniques that i'll use in the model. Then design the class for handle the data generator, and finaly implement the model with all the good stuffs.</h3>

<h3 style="text-align:center">Walking throw the competition notebooks, i've learn a lot from others and  found that many folks made his own dataset for cassava challenge, they did merge dataset from 2019 and 2020, many just image and other using tfrecord. So, i've taken one of those, this dataset , i really recomended.</h3>

<h3 style="text-align:center">If you have any sugestion or observation, please let me know in the coments bellow.</h3>


<h2 style="text-align:center; font-family:'Lobster', cursive">Let's do it!</h2>

# Table of Contents
1. [Chapter 1 - What is the problem?](#ch1)
1. [Chapter 2 - About our tools](#ch2)
1. [Chapter 3 - Overview data](#ch3)
1. [Chapter 4 - Exploring Image Augmentation](#ch4)
1. [Chapter 5 - Data Generator](#ch5)
1. [Chapter 6 - Model](#ch6)
1. [Chapter 7 - Training the base Model](#ch7)
1. [Chapter 8 - Training Kfold Model](#ch8)
1. [Chapter 9 - Submission - Test](#ch9)
1. [Submission File](#ch99)

In [ ]:
"""
import os
cont = 0
for dirname, _, filenames in os.walk('kaggle_data/train'):
  for filename in filenames:
    cont= cont +1
    #print(os.path.join(dirname, filename))
assert 26337 <= cont ,"There are not enough images files, only {} files".format(cont)
print("Number of images : {}".format(cont))
"""

In [ ]:
!pip install --quiet ../input/kerasapplications
!pip install --quiet ../input/efficientnet-source-code

In [ ]:
#!pip install -q efficientnet

In [ ]:
# import libraries
import numpy as np 
import pandas as pd 
from os.path import join
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import os

import IPython.display as display

from PIL import Image
import warnings
#warnings.filterwarnings("ignore")

import random as rd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Input,optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization, Activation, Input, GlobalAveragePooling2D, concatenate,Lambda
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping,TensorBoard
from tensorflow.keras.applications import EfficientNetB0,EfficientNetB4,EfficientNetB3, Xception
import efficientnet.tfkeras as efn

from tensorflow.keras.mixed_precision import experimental as mixed_precision
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,roc_curve,multilabel_confusion_matrix
from sklearn.metrics import classification_report
from tqdm import tqdm
import itertools

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
def plot_confusion_matrix(y_true, y_pred, class_names,title="Confusion matrix",normalize=False,onehot = False):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.
    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    if onehot :
        cm = confusion_matrix([y_i.argmax() for y_i in y_true], [y_ip.argmax() for y_ip in y_pred])
    else:
        cm = confusion_matrix(y_true, y_pred)
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2) if normalize else cm

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

<a id="ch1"></a>
# CHAPTER 1. What is the problem?

Cassava leaf, like any other plant is vulnerable to diseases and if you have a sick leaf, then this disease will be going to propagate to others leaf and potentially spread the disease for the whole plantation. So, we need a cassava leaf desease expert at the plantation (that will be expensive) or create an image classifier to detect automatically if there are any of the 4 diseases or if the leaf is healthy.

Can you identify a problem with a cassava plant using a photo from a relatively inexpensive camera? This competition will challenge you to distinguish between several diseases that cause material harm to the food supply of many African countries. In some cases the main remedy is to burn the infected plants to prevent further spread, which can make a rapid automated turnaround quite useful to the farmers.

<a id="ch2"></a>
# CHAPTER 2. About our tools?

1. [EfficientNET](https://github.com/qubvel/efficientnet)
1. [Image aumentation](https://www.tensorflow.org/tutorials/images/data_augmentation)
1. [TensorFlow Hub - Efficientnet](https://tfhub.dev/google/collections/efficientnet/1)
1. [Noisy student paper - Efficienet improve ](https://arxiv.org/pdf/1911.04252.pdf)
1. [Merge Dataset Cassava challenge 2019 and 2020](https://www.kaggle.com/tahsin/cassava-leaf-disease-merged)

<center>
    <img src= "https://raw.githubusercontent.com/tensorflow/tpu/master/models/official/efficientnet/g3doc/params.png" width="400">
</center>

<a id="ch3"></a>
# CHAPTER 3. Overview Data

### Paths 📚

In [ ]:
SampleSubmission = "../input/cassava-leaf-disease-classification/sample_submission.csv"
labels_desease  = "../input/cassava-leaf-disease-classification/label_num_to_disease_map.json"
traintablepath = "../input/cassava-leaf-disease-merged/merged.csv"

train_tfr_path = "../input/cassava-leaf-disease-classification/train_tfrecords"
test_tfr_path = "../input/cassava-leaf-disease-classification/train_tfrecords"

train_img_path = "../input/cassava-leaf-disease-merged/train"
test_img_path  = "../input/cassava-leaf-disease-classification/test_images"

models_path  = ""

### Open files 📚

In [ ]:
train_table = pd.read_csv(traintablepath)
classes_label= pd.read_json(labels_desease, orient='index')
sampSum = pd.read_csv(SampleSubmission)

## Train file overview

1. Sample train table, only five rows.
1. Because this is a deeplearning problem, our train table doesn't have any issue
1. We need to analyse how many classes there are in our dataset, and how they are introduce to us, like hot encoder or numerical.
1. There are only one type of deseasse by image, we have 21397 uniques images and 21397 labeled data points.

In [ ]:
display.display(train_table.info())
display.display(train_table.nunique())
train_table.head()

### Desease Numerical class ☠️

In [ ]:
classes_label = classes_label.reset_index()
classes_label.columns = ["label", "Desease"]
classes_label["short"] = classes_label["Desease"].apply(lambda x: x.split(" ")[-1])
display.display(classes_label)

### Diseases representation 💀

In [ ]:
temp = train_table.join(classes_label[["label", "short"]].set_index('label'), on= "label")[["label", "short"]]
temp["count"] = 1
temp = temp.groupby(["label", "short"]).sum()
class_count = temp.reset_index()
fig = px.bar(class_count, x="short", y="count", title="Count per leaf desease",text=class_count['count'])
fig.update_traces(texttemplate='%{text:.2s}',textposition='outside')
fig.update_layout(template= "plotly_dark" , 
                  xaxis = dict(title = "Cassava Leaf Desease"))
fig.show()

### Weighting classes  🏃  🏃  🏃  🏃 

Because this dataset is imbalanced, I will need to consider create the inverse weight to get better performance on my model.

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_table["label"]),
                                                 train_table["label"])
class_weights = dict(enumerate(class_weights)) 
display.display(class_weights)

<a id="ch4"></a>
# Chapter 4 - Exploring Image Augmentation

# Exploring Images

In [ ]:
def print_leaf(path_folder, table, augg_funct=None, augg = False,prefix="Orig" ):
    temp = table.iloc[[rd.randint(0, len(train_table)) for _ in range(6)]]
    
    plt.figure(figsize=(15,9))
    for i, (name, label )in enumerate(zip(temp["image_id"], temp["label"])):
        plt.subplot(2,3, i%6 +1)
        plt.axis('off')
        if not augg:
            img=Image.open(join(path_folder,name))
        else:
            image = tf.expand_dims(np.array(Image.open(join(path_folder,name))), 0)
            img=augg_funct(image)[0]
        plt.imshow(img)
        img_size = np.asarray(img).shape
        plt.title("{} {} - size {} ".format(prefix,classes_label["short"][int(label)], img_size) )

def print_leaf_mix_cutup(x,y,prefix ):
    plt.figure(figsize=(15,9))
    for i, (image, label )in enumerate(zip(x,y)):
        if i > 5:
          continue
        plt.subplot(2,3, i%6 +1)
        plt.axis('off')
        img=image.numpy()
        plt.imshow(img)
        img_size = np.asarray(img).shape
        plt.title("{} {} - size {} ".format(prefix,classes_label["short"][np.where(label>0)[0].tolist()].tolist(), img_size) )

In [ ]:
print_leaf(train_img_path, train_table)

## Exploring Image Aumentation

Tensorflow keras, provide us a set of tools and techniques that we can use direct form the box like
any other layer. On this proyect y going to use 6 types of preprocessing techniques.
1. [RandomCrop](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/RandomCrop)
    Randomly crop the images to target height and width.
1. [RandomTranslation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/RandomTranslation)
    Randomly translate each image during training.
1. [RandomZoom](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/RandomZoom)
    Randomly zoom each image during training.
1. [RandomFlip](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/RandomFlip)
    Randomly flip each image horizontally and vertically.
1. [RandomRotation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/RandomRotation)
    Randomly rotate each image
1. [RandomContrast](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/RandomContrast)
    Adjust the contrast of an image or images by a random factor.

In [ ]:
img_augmentation = tf.keras.Sequential(
    [
        #tf.keras.layers.experimental.preprocessing.RandomCrop(image_size, image_size), #during inference will work like a resize layer
        #tf.keras.layers.experimental.preprocessing.RandomTranslation((-.1,.1), (-.1,-.1)),
        tf.keras.layers.experimental.preprocessing.RandomZoom((-.1,-.1)),
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
        tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),
    ])

In [ ]:
print_leaf(train_img_path, train_table,img_augmentation,True, prefix= "Augm-" )

<a id="ch4.1"></a>
# Chapter 4.1 - CutMix and MixUp

Acknowledge to @cdotte Chris Deotte for his amazing [notebook](https://www.kaggle.com/cdeotte/cutmix-and-mixup-on-gpu-tpu)

### CutMix Augmentation
You can see more about cutmix [here](https://arxiv.org/abs/1905.04899)

Cut a portion form an images and paste randomly into another image.
The same with the labels. 

### MixUp Augmentation
You can see more about mixup [here](https://arxiv.org/abs/1710.09412)

______________________________
#### Both methods use the same logic, lambda determines how much to mix
```
X_i1 = tensor_images
y_i1 = label

X_i2 = tensor_images
y_i2 = label

lambda = beta_distribution

x_cutmix = x_i1*lambda + x_i2*(1-lambda)
y_cutmix = y_i1*lambda + y_i2*(1-lambda)
```

In [ ]:
class ImageCustomGenerator(tf.keras.utils.Sequence):
    """
     df_table, dataframe
     col_name, column name for file images
     col_label, column label
     indexs, index to extract from dataframe
     path_files, folder path where images are stored
     batch_size,
     target_size, output images
    """
    def __init__(self, 
                 df_table,
                 col_name,
                 col_label,
                 path_files,
                 batch_size,
                 target_size,
                 augmented_seq=None,
                 indexs=None,
                 shuffle = False,
                 TEST = False
                ):
        self.df_table = df_table
        self.augmented_seq = augmented_seq
        self.indexs = indexs if not TEST else df_table.index.tolist()
        self.col_name = col_name
        self.col_label = col_label
        self.path_files = path_files
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.TEST = TEST
        self.n_class = df_table[col_label].nunique() if not TEST else 1
        self.on_epoch_end()
        self.__info()
        
    def __info(self):
        print("Custom Image generator {:4d} images".format(len(self.indexs)))

    def __len__(self):
        #batches per epochs
        return len(self.indexs) // self.batch_size + 1 if len(self.indexs)%self.batch_size != 0 else len(self.indexs) // self.batch_size

    def __getitem__(self, index):
        indexs_step = self.index_gen[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indexs[k] for k in indexs_step]
        X, y = self.__get_data(batch)
        return X, y

    def on_epoch_end(self):
        self.index_gen = np.arange(len(self.indexs))
        if self.shuffle == True and self.TEST == False:
            np.random.shuffle(self.index_gen)
            
    def __get_data(self, batch):
        X = []
        if not self.TEST:
            y = self.df_table.iloc[batch,:][self.col_label].to_numpy().astype(int)
            file_list = self.df_table.iloc[batch,:][self.col_name]
            file_list = file_list.map(lambda x: os.path.join(self.path_files,x))

            for name in file_list:
                img_ = tf.image.resize(np.array(np.array(Image.open(name))), self.target_size, method = "bilinear")
                X.append(img_)
            return self.transform(tf.stack(X), tf.stack(y))
        
        else:#################### TEST BEHAVIOR ################
            file_list = self.df_table.iloc[batch,:][self.col_name]
            file_list = file_list.map(lambda x: os.path.join(self.path_files,x))
            for name in file_list:
                img_ = tf.image.resize(np.array(np.array(Image.open(name))),[600,800], method = "bilinear")
                img_ = tf.image.random_crop(img_, [*self.target_size,3])
                X.append(img_)
            return  self.augmented_seq(tf.stack(X))/255,  tf.zeros([len(batch)], tf.float32)
            
    
        
    def cutmix(self, image, label, PROBABILITY = 1.0):
        # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
        # output - a batch of images with cutmix applied
        IMAGE_SIZE = self.target_size
        CLASSES = self.n_class
        AUG_BATCH = self.batch_size
        DIM = IMAGE_SIZE[0]
        
        imgs = []; labs = []
        for j in range(AUG_BATCH):
            # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
            P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
            # CHOOSE RANDOM IMAGE TO CUTMIX WITH
            k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
            # CHOOSE RANDOM LOCATION
            x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
            y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
            b = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0
            WIDTH = tf.cast( DIM * tf.math.sqrt(1-b),tf.int32) * P
            ya = tf.math.maximum(0,y-WIDTH//2)
            yb = tf.math.minimum(DIM,y+WIDTH//2)
            xa = tf.math.maximum(0,x-WIDTH//2)
            xb = tf.math.minimum(DIM,x+WIDTH//2)
            # MAKE CUTMIX IMAGE
            one = image[j,ya:yb,0:xa,:]
            two = image[k,ya:yb,xa:xb,:]
            three = image[j,ya:yb,xb:DIM,:]
            middle = tf.concat([one,two,three],axis=1)
            img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
            imgs.append(img)
            # MAKE CUTMIX LABEL
            a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
            if len(label.shape)==1:
                lab1 = tf.one_hot(label[j],CLASSES)
                lab2 = tf.one_hot(label[k],CLASSES)
            else:
                lab1 = label[j,]
                lab2 = label[k,]
            labs.append((1-a)*lab1 + a*lab2)

        # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
        image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
        label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
        return image2,label2
    

    def mixup(self,image, label, PROBABILITY = 1.0):
        # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
        # output - a batch of images with mixup applied
        IMAGE_SIZE = self.target_size
        CLASSES = self.n_class
        AUG_BATCH = self.batch_size
        
        DIM = IMAGE_SIZE[0]

        imgs = []; labs = []
        for j in range(AUG_BATCH):
            # DO MIXUP WITH PROBABILITY DEFINED ABOVE
            P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.float32)
            # CHOOSE RANDOM
            k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
            a = tf.random.uniform([],0,1)*P # this is beta dist with alpha=1.0
            # MAKE MIXUP IMAGE
            img1 = image[j,]
            img2 = image[k,]
            imgs.append((1-a)*img1 + a*img2)
            # MAKE CUTMIX LABEL
            if len(label.shape)==1:
                lab1 = tf.one_hot(label[j],CLASSES)
                lab2 = tf.one_hot(label[k],CLASSES)
            else:
                lab1 = label[j,]
                lab2 = label[k,]
            labs.append((1-a)*lab1 + a*lab2)

        # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
        image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
        label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
        return image2,label2

    def transform(self,image,label):
        # THIS FUNCTION APPLIES BOTH CUTMIX AND MIXUP
        DIM = self.target_size[0]
        CLASSES = self.n_class
        AUG_BATCH = self.batch_size
        
        SWITCH = 0.5
        CUTMIX_PROB = 0.666
        MIXUP_PROB = 0.666
        # FOR SWITCH PERCENT OF TIME WE DO CUTMIX AND (1-SWITCH) WE DO MIXUP
        image2, label2 = self.cutmix(image, label, CUTMIX_PROB)
        image3, label3 = self.mixup(image, label, MIXUP_PROB)
        imgs = []; labs = []
        for j in range(AUG_BATCH):
            P = tf.cast( tf.random.uniform([],0,1)<=SWITCH, tf.float32)
            imgs.append(P*image2[j,]+(1-P)*image3[j,])
            labs.append(P*label2[j,]+(1-P)*label3[j,])
        # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
        image4 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
        label4 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
        return image4,label4

In [ ]:
migen_test = ImageCustomGenerator(
                 df_table = train_table,
                 col_name = "image_id",
                 col_label = "label",
                 augmented_seq = img_augmentation,
                 path_files=train_img_path ,
                 batch_size = 6,
                 target_size = (450, 450),
                 TEST = True,
                 
)
x, y = migen_test.__getitem__(2)
print_leaf_mix_cutup(x,y,"Test Format")

<a id="ch5"></a>
# Chapter 5 - Data Generator

OOF require that we create a new data generator for every fold. So, this image data generator take that idea.


In [ ]:
def img_data_generator(df,
                       col_name, 
                       col_label, 
                       train_index, 
                       val_index,
                       path_files,
                       df_test = None,
                       path_test_files= None,
                       augmented_seq = None,
                       target_size=(300,300),
                       batch_size=32, 
                       seed=10):
    data_gen_args = dict(
                     rotation_range=20,
                     fill_mode = "reflect",
                     horizontal_flip = True,
                     vertical_flip = True,
                     width_shift_range=0.05,
                     height_shift_range=0.05,
                     shear_range = .1,
                     zoom_range=0.1)
    
    train_datagen = ImageCustomGenerator(
                        df_table = df,
                        col_name = col_name,
                        col_label = col_label,
                        indexs = train_index,
                        path_files=path_files ,
                        batch_size = batch_size,
                        target_size = target_size,)
    
    # train_datagen = ImageDataGenerator().flow_from_dataframe(
    #                   dataframe = df.iloc[train_index],
    #                   directory= path_files,
    #                   x_col=col_name,
    #                   y_col=col_label,
    #                   color_mode="rgb",
    #                   target_size=target_size,
    #                   batch_size=batch_size,
    #                   shuffle=True,
    #                   class_mode='categorical',
    #                   validate_filenames=True,
    #                   seed=seed)

    val_datagen = ImageDataGenerator().flow_from_dataframe(
                dataframe = df.iloc[val_index],
                directory = path_files,
                x_col=col_name,
                y_col=col_label,
                color_mode="rgb",
                target_size=target_size,
                batch_size=batch_size,
                shuffle=False,
                class_mode='categorical',
                validate_filenames=True,
                seed=seed)
    if df_test is not None :
        test_datagen = ImageCustomGenerator(
                        df_table = df_test,
                        col_name = col_name,
                        col_label = col_label,
                        augmented_seq = augmented_seq,
                        path_files=path_test_files ,
                        batch_size = batch_size,
                        target_size = target_size,
                        TEST = True,
                        )
    else:
        test_datagen = None
    return train_datagen, val_datagen, test_datagen

<a id="ch6"></a>
# Chapter 6 - Model Creation

I will create a class for my model, this class have all the good stuff for a deeplearing model, transfern learning model.

The class create de model, manage training and change from static pretraining models to whole trainable model.

I've considered three principals call backs,
1. **ModelCheckpoint** : save my progress
1. **EarlyStopping** : Save time
1. **ReduceLROnPlateau** : Avoid early stoping and keep improving the model

### Learing Rate Scheduler

I've saw this lr scheduler [here](https://colab.research.google.com/notebooks/tpu.ipynb#scrollTo=iXJXG-Ufdbnu) and in so many notebooks here on kaggle. Using for pretrained models. 


In [ ]:
def lrfn(epoch):
    """
    Learing rate schedule for pretrained models
    """
    start_lr = 0.00001
    min_lr = 0.00001
    max_lr = 0.00005 * 3
    rampup_epochs = 2
    sustain_epochs = 0
    exp_decay = .35
    if epoch < rampup_epochs:
      return (max_lr - start_lr)/rampup_epochs * epoch + start_lr
    elif epoch < rampup_epochs + sustain_epochs:
      return max_lr
    else:
      return (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr


In [ ]:
n_epochs = 6
rang = np.arange(n_epochs)
y = [lrfn(x) for x in rang]
plt.plot(rang, y,linewidth=2, color = "blue")
plt.xlabel("N° epoch")
plt.ylabel("LR")
plt.grid(True)
_ = plt.title('Learning rate per epoch:')

In [ ]:
class cassava_model(object):
    def __init__(self, 
                 input_shape,
                 n_classes, 
                 augment_layer, 
                 lr=1e-4, 
                 saved_file = None,
                 model_name= "cassavaleaf",
                 model_suff = 0,
                 tb_file = "log1",
                 metric_name = False,
                 ):
        self.input_shape = input_shape
        self.n_classes = n_classes
        self.name = model_name
        self.tb_file = tb_file
        self.name_suff = model_suff
        self.augm_layer = augment_layer # layer for image augmentation
        self.saved_file = saved_file
        self.raw_name = model_name + "_" + str(model_suff) + '.hdf5'
        self.metric_name = metric_name
        self.n_epochs = 0
        self.lr = lr
        self.create_model()
        self.load_weights() if saved_file is not None else None
        
    def create_model(self,):
        input1 = Input(self.input_shape)
        aug_input = self.augm_layer(input1)
        x = Lambda(lambda x: x/255)(aug_input)
      
        x = efn.EfficientNetB5(weights = "noisy-student", include_top = False )(x)
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.2)(x)

        z = Dense(5, activation="softmax", dtype='float32')(x)
        self.model = Model(inputs=input1, outputs=z)
        self.compile_model()
        
    def compile_model(self,lr=None):
        """
        MODEL compile, UTIL AFTER ANY CHANGE IN THE MODEL
        """
        if lr is not None:
          self.lr = lr
        self.model.summary()
        self.model.compile(loss='categorical_crossentropy',
                      optimizer= optimizers.Adam(learning_rate=self.lr),
                      metrics=[tf.keras.metrics.CategoricalAccuracy(),
                               tf.keras.metrics.PrecisionAtRecall(recall=0.8),
                               tf.keras.metrics.AUC(name='auc', multi_label= True),
                               tf.keras.metrics.Recall(name='recall')])

    def load_weights(self,model_path_file=None):
        """
        LOAD THE MODEL BY PATH
        """
        if (self.saved_file) or model_path_file:
            try:
                #model.load_model(saved_file)
                self.model.load_weights(self.saved_file if self.saved_file else model_path_file )
                print("Success : Loaded model weights")
            except:
                print("Fail: Weights not loaded")
                
    def fit_model(self,**kwarg):
        """
        TRAIN THE MODEL
        logdir: log dir for tensorboard files
        epoch_add: number of epochs to train
        train_data: datagen for the trainig
        val_data: datagen for validation
        
        """
        logdir= kwarg["logdir" ]
        epoch_add = kwarg["epoch_add" ]
        file2save = str(self.name) + "_" + str(self.name_suff) 
        if self.metric_name :
            file2save = file2save + "_" +'{val_loss:.4f}acc_val{val_categorical_accuracy:.4f}acc_train{categorical_accuracy:.4f}.hdf5'
        else: 
            file2save = file2save + '.hdf5'

        callbacks_ = []
        callbacks_.append( TensorBoard(log_dir=self.tb_file))
        #csv_logger = CSVLogger('training.log')
        #callbacks.append( reduce_LR = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=2,))
        callbacks_.append( ModelCheckpoint( file2save , monitor='val_loss',verbose=1, save_best_only=True) )
        callbacks_.append( EarlyStopping(monitor='val_loss',patience=2,min_delta=0))
        callbacks_.append( tf.keras.callbacks.LearningRateScheduler(lambda epoch: self.lrfn(epoch), verbose=True)) if type(self.lr) == float else None
    
        self.model.fit(kwarg["train_data" ],
                    #steps_per_epoch = 8,   #
                    #batch_size=6,          #                    
                    epochs=self.n_epochs+kwarg["epoch_add" ],
                    initial_epoch = self.n_epochs,
                    callbacks=callbacks_,        
                    validation_data = kwarg["val_data" ],
                    class_weight = kwarg["class_weight" ],
                    workers=-1)
        self.n_epochs=self.n_epochs+epoch_add
    @staticmethod 
    def lrfn(epoch):
        """
        Learning rate schedule applied to pretrained models
        Design for 6 epochs, fresh train
        """
        start_lr = 0.00001
        min_lr = 0.00001
        max_lr = 0.00005 * 3
        rampup_epochs = 2
        sustain_epochs = 0
        exp_decay = .35

        if epoch < rampup_epochs:
          return (max_lr - start_lr)/rampup_epochs * epoch + start_lr
        elif epoch < rampup_epochs + sustain_epochs:
          return max_lr
        else:
          return (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr


<a id="ch7"></a>
# Chapter 7. Training the Base - Model

This will be the main training, for stablish the first training weights. Thes best model posible.

In [ ]:
from sklearn.model_selection import train_test_split
train_table["label"] = train_table["label"].astype(str)
sampSum["label"] = sampSum["label"].astype(str)
index_train, index_val, _, _ = train_test_split(list(train_table.index),
                                            train_table["label"], 
                                            test_size=0.20,
                                            random_state =50,
                                            stratify= train_table["label"])

target_size = (456, 456) # recomended size for EffientnetB5, please refer to the links in the beggining.

### Training Parameters

In [ ]:
"""
train_param_data = {
                    "df": train_table,
                    "col_name": "image_id",
                    "col_label": "label",
                    "train_index": index_train, ### change for every kfold
                    "val_index": index_val,     ### change for every kfold
                    "path_files":train_img_path,
                    "batch_size":6,
                    "target_size": target_size,
                    "seed": 16,
                    "df_test": sampSum,
                    "path_test_files": test_img_path, 
}
train, val, test = img_data_generator(**train_param_data)
fit_params = {
            "logdir": "log2",
            "epoch_add": 3,
            "train_data": train,
            "val_data": val,
            "class_weight": class_weights,
}
"""

In [ ]:
"""
tf.keras.backend.clear_session()
model = cassava_model(input_shape = (*target_size,3),n_classes= 5, augment_layer=img_augmentation,saved_file = "k_model_0.hdf5")
"""

### Training Time

In [ ]:
#model.fit_model(**fit_params)

### Evaluation Metrics

In [ ]:
"""
val_predict = model.model.predict(val, verbose=1) 
test_labels = pd.get_dummies(train_table.iloc[index_val]["label"]).to_numpy()
plot_confusion_matrix(test_labels, val_predict,classes_label["short"],normalize=False, onehot= True)
print(classification_report([y_i.argmax() for y_i in test_labels], 
                      [y_ip.argmax() for y_ip in val_predict],
                      digits=4 ) )
"""

# Chapter 8 - Training Kfold Model

### General RUN

In [ ]:
"""
tf.keras.backend.clear_session() 
sampSum["label"] = sampSum["label"].astype(str)
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits)
k_ready = [0,1]
for ind_k, (index_train, index_val) in enumerate(skf.split(train_table["image_id"], train_table["label"])):
    if ind_k in k_ready:
        continue
    # Creating generators
    train_param_data["train_index"]= index_train
    train_param_data["val_index"]= index_val
    train_param_data["batch_size"]= 6
    train, val, _ = img_data_generator(**train_param_data)

    # Creating the k model
    model_params = {
                "input_shape": (*target_size,3),
                "n_classes" : 4,
                "augment_layer": img_augmentation,
                "model_name": "k_model",
                "model_suff": ind_k,
                "tb_file": "logk_" + str(ind_k),
                "saved_file": "k_model" + "_" + str(ind_k)  + '.hdf5',
    }
    model = cassava_model(**model_params)

    # Creating Training parameters
    fit_params = {
                "logdir": "log_" + str(ind_k),
                "epoch_add": 4,
                "train_data": train,
                "val_data": val,
                "class_weight": class_weights,
    }
    ###### Automatic best model and tensorboard saved #########
    # Training the k model - first round , lr linear incremental then exp decay
    model.fit_model(**fit_params)  

    # Training the k model - second round, lr cosine decay
    fit_params["epoch_add"] = 4
    lrcos = tf.keras.experimental.CosineDecay(initial_learning_rate=5e-5, 
                                       decay_steps=int(len(train))*fit_params["epoch_add"] )
    model.compile_model(lr=lrcos)
    model.fit_model(**fit_params) 
"""

# Chapter 9 - Submission - Test

In [ ]:
def create_model(model_id,CLASSES=5,IMAGE_SIZE=(450,450)):
  if model_id==0:
    pretrained_model = tf.keras.applications.Xception(weights = None, input_shape=[*IMAGE_SIZE, 3], include_top=False)
  elif model_id==1:
    pretrained_model = efn.EfficientNetB5(weights = None, include_top = False,input_shape=[*IMAGE_SIZE, 3] )
  elif model_id ==2:
    pretrained_model = efn.EfficientNetB0(weights = None, include_top = False,input_shape=[*IMAGE_SIZE, 3] )
  
  pretrained_model.trainable = True
  model = tf.keras.Sequential([
    pretrained_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(CLASSES, activation='softmax')
  ])
  model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=[tf.keras.metrics.CategoricalAccuracy(),
                               tf.keras.metrics.PrecisionAtRecall(recall=0.8),
                               tf.keras.metrics.AUC(name='auc', multi_label= True),
                               tf.keras.metrics.Recall(name='recall')])
  return model

In [ ]:
%time
import glob
path_model_efnB5 = glob.glob("../input/cassava-efficientnetb5-450px/*.hdf5")
path_model_xcept = glob.glob("../input/cassava-xception-450px/*.hdf5")
path_model_efnB0 = glob.glob("../input/cassava-efficientnetb0-450/*.hdf5")

TTA_n = 4

#table_test_dummy = sampSum.copy()
table_test_dummy = pd.DataFrame()
table_test_dummy['image_id'] = list(os.listdir(test_img_path))

n_splits = 5
#### Justo for testing purpose
test_gen = ImageCustomGenerator(
                 df_table = table_test_dummy, # sampSum
                 col_name = "image_id",
                 col_label = "label",
                 augmented_seq = img_augmentation, 
                 path_files= test_img_path ,       # test_img_path
                 batch_size = 6,
                 target_size = (450, 450),
                 TEST = True,
                 )
y_predict = np.zeros((len(table_test_dummy),5))     #np.zeros((len(sampSum),5))
model_name = ["Xception", "EfficientNet B5", "EfficientNet B0"]
for id_model, list_kfold_models in enumerate([path_model_xcept, path_model_efnB5,path_model_efnB0 ]):
    y_predict_model = np.zeros((len(table_test_dummy),5))
    print("Inference by model - {}".format(model_name[id_model]))
    for ki, model_kfold_path in enumerate(list_kfold_models):
        print("{} - {} Kfold pre trained".format(model_name[id_model], ki))
        tf.keras.backend.clear_session() 
        model = create_model(id_model)
        model.load_weights(model_kfold_path)
        y_predict_tta = np.zeros((len(table_test_dummy),5))
        for t in range(TTA_n):
            print("N°{} TTA".format(t))
            y_predict_tta += model.predict(test_gen, verbose=1)
            
        y_predict_model += y_predict_tta/TTA_n
    y_predict += y_predict_model/5*.333

    
y_predict = np.argmax(y_predict, axis = 1)
table_test_dummy["label"] = y_predict
table_test_dummy.to_csv("submission.csv", index=False)
table_test_dummy.head()